In [ ]:
!pip show pandas

In [ ]:
!pip install jupyter ipywidgets

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install tabulate matplotlib pandas

In [63]:
path_to_project = '/home/wsl/git/Wf-Security/heuristic_and_exact_solutions'
output_folder = 'output'
output_file_name = 'wf_security_executions.csv'

toy_instances = [
    "Synthetic_005_A", 
    "Synthetic_005_B", 
    "Synthetic_005_C", 
    "Synthetic_010_A", 
    "Synthetic_010_B", 
    "Synthetic_010_C",
    "Synthetic_015_A", 
    "Synthetic_015_B", 
    "Synthetic_015_C"
]

instances = [
    "Synthetic_005_A", 
    "Synthetic_005_B", 
    "Synthetic_005_C", 
    "Synthetic_010_A", 
    "Synthetic_010_B", 
    "Synthetic_010_C",
    "Synthetic_015_A", 
    "Synthetic_015_B", 
    "Synthetic_015_C"
    "CyberShake_030", 
    "CyberShake_050", 
    "CyberShake_100", 
    "Epigenomics_024", 
    "Epigenomics_046", 
    "Epigenomics_100",
    "Inspiral_030", 
    "Inspiral_050", 
    "Inspiral_100",
    "Montage_025", 
    "Montage_050", 
    "Montage_100",
    "Sipht_030", 
    "Sipht_060", 
    "Sipht_100"
]

real_instances = [
    "CyberShake_030", 
    "CyberShake_050", 
    "CyberShake_100", 
    "Epigenomics_024", 
    "Epigenomics_046", 
    "Epigenomics_100",
    "Inspiral_030", 
    "Inspiral_050", 
    "Inspiral_100",
    "Montage_025", 
    "Montage_050", 
    "Montage_100",
    "Sipht_030", 
    "Sipht_060", 
    "Sipht_100"
]

scenarios = [
#    1,
#    2,
#    3,
    4
]

algorithms = [
    'grch',
    'grasp',
    'heft'
]


In [92]:
import os

def listar_pastas(diretorio):
    # Verificar se o diretório existe
    if not os.path.isdir(diretorio):
        print(f'O diretório "{diretorio}" não existe.')
        return []

    # Lista para armazenar os nomes das pastas
    pastas = []

    # Iterar sobre os arquivos e pastas no diretório
    for nome in os.listdir(diretorio):
        caminho = os.path.join(diretorio, nome)
        if os.path.isdir(caminho):
            pastas.append(nome)
    pastas.sort(reverse=True)
    return pastas

# Exemplo de uso:
folders = listar_pastas(f'{path_to_project}/{output_folder}')
# print("Pastas encontradas:")
# for pasta in pastas:
#     print(pasta)


In [93]:
import ipywidgets as widgets

selected_option = folders[0] if folders else ''

# Opções para o menu suspenso
# options = ['Opção 1', 'Opção 2', 'Opção 3']

# Criar o widget Dropdown
dropdown = widgets.Dropdown(options=folders, layout={'width': '200px'})

# Definir a largura do rótulo
dropdown_label = widgets.Label('Output:')

# selected_option = ''
# Função para lidar com a seleção
def handle_dropdown_change(change):
    global selected_option
    selected_option = change.new
    print(f'Opção selecionada: {selected_option}')

# Registrar a função de manipulação de eventos
dropdown.observe(handle_dropdown_change, names='value')

# Exibir o rótulo e o dropdown
display(widgets.HBox([dropdown_label, dropdown]))

# Exibir o widget
# display(dropdown)
# display(widgets)

In [8]:
import pandas as pd

# path_to_project = '/home/wsl/git/Wf-Security/heuristic_and_exact_solutions'
# output_file_name = 'wf_security_executions.csv'


# Ler o arquivo CSV
df = pd.read_csv(f'{path_to_project}/{output_folder}/{selected_option}/{output_file_name}')

# Listar os diferentes valores de um campo específico (por exemplo, 'campo_alvo')
available_algorithms = df['algorithm'].unique()

# Exibir os diferentes valores
print(available_algorithms)

['grasp']


In [11]:
import ipywidgets as widgets

selected_available_algorithms = available_algorithms[0] if folders else ''

# Opções para o menu suspenso
# options = ['Opção 1', 'Opção 2', 'Opção 3']

# Criar o widget Dropdown
dropdown = widgets.Dropdown(options=available_algorthms, layout={'width': '200px'})

# Definir a largura do rótulo
dropdown_label = widgets.Label('Algorithm:')

# selected_option = ''
# Função para lidar com a seleção
def handle_available_algorithms_dropdown(change):
    global selected_available_algorithms
    selected_available_algorithms = change.new
    print(f'Selected Algorithm: {selected_option}')

# Registrar a função de manipulação de eventos
dropdown.observe(handle_dropdown_change, names='value')

# Exibir o rótulo e o dropdown
display(widgets.HBox([dropdown_label, dropdown]))

In [16]:
import csv

# Lista para armazenar as variações únicas
variacoes = set()

# Lendo o arquivo CSV
with open(f'{path_to_project}/{output_folder}/{selected_option}/{output_file_name}', 'r') as file:
    reader = csv.reader(file)
    
    # Lendo o cabeçalho para obter os nomes dos campos
    header = next(reader)
    
    # Obter os índices dos campos desejados
    indice_campo4 = header.index('alpha_time')
    indice_campo5 = header.index('alpha_cost')
    indice_campo6 = header.index('alpha_security')
    indice_campo7 = header.index('alpha_restricted_candidate_list')
    
    # Iterar pelas linhas do arquivo
    for row in reader:
        # Extrair os valores dos 4 primeiros campos
        campo1, campo2, campo3, campo4, *_ = row[3:7]
        
        # Adicionar a variação única à lista
        variacoes.add((campo1, campo2, campo3, campo4))

# Ordenar as variações únicas
variacoes_ordenadas = sorted(variacoes, reverse=True)

# Criar o dropdown com as variações únicas
from ipywidgets import Dropdown

dropbox = Dropdown(
    options=variacoes_ordenadas,
    description='Alphas:'
)
dropbox

Dropdown(description='Alphas:', options=(('0.9', '0.05', '0.05', '0.3'), ('0.05', '0.9', '0.05', '0.3'), ('0.0…

In [52]:
print(selected_option)
display(selected_option)

20240106213947


'20240106213947'

In [ ]:
import ipywidgets as widgets

# Criando um botão
botao = widgets.Button(description="Clique aqui")

# Função a ser chamada quando o botão é clicado
def on_button_clicked(b):
    # display("Botão clicado!")
    print("Botão clicado!")

# Conectando a função ao evento de clique do botão
botao.on_click(on_button_clicked)

# Exibindo o botão
display(botao)

# Criando um campo de entrada de texto
entrada = widgets.Text()

# Função a ser chamada quando o texto é alterado
def handle_text_change(text):
    print("Texto alterado para:", text)

# Conectando a função ao evento de alteração do texto
entrada.observe(handle_text_change, names='value')

# Exibindo o campo de entrada de texto
display(entrada)

In [ ]:
# Importing the required modules.
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import figure
from tabulate import tabulate

# Read data.
# df = pd.read_csv(f'wf_security_executions.csv', header='infer')
# df = pd.read_csv(f'wf_security_executions_w_grasp.csv', header='infer')
df = pd.read_csv(f'{path_to_project}/{output_folder}/{selected_option}/{output_file_name}', header='infer')

# alpha_time,alpha_Cost,alpha_security

# print(df[df['instance'].str.startswith('Synthetic_')])
# type(df['instance'])

# print(newdf)

# dir(df)
# print(df)

for instance in instances:
    print('Início da instância', instance)
    for scenario in scenarios:
        print('Início do scenario', scenario)
        for algorithm in algorithms:
            newdf = df[(1==1)
                       & (df['algorithm'] == algorithm)
                       # & (df.alpha_time == 0.90) 
                       # & (df.alpha_cost == .05) 
                       # & (df.alpha_security == .05) 
                       & (df['instance'].str.startswith(instance))
                       & (df.scenario == scenario)
                       & (df.num_buckets == 16)
                      ]
            print(newdf)
            # Selecione apenas as colunas numéricas
            colunas_numericas = newdf.select_dtypes(include=['float64', 'int64']).columns
            # print(colunas_numericas)
            colunas_desejadas = [
                'num_iterations',
                'object_function',
                'makespan',
                'cost',
                'security',
                'time',
                'best_solution_iteration',
                'best_solution_time',
                'lsn_time_1',
                'lsn_noi_1',
                'lsn_time_2',
                'lsn_noi_2',
                'lsn_time_3',
                'lsn_noi_3'
            ]
            print(' '.join(map(str, colunas_desejadas)))
            # Filtra os valores positivos em cada coluna desejada
            # for coluna in colunas_desejadas:
            #     newdf[coluna] = newdf[coluna].apply(lambda x: x if x >= 0 else None)
            for coluna in colunas_desejadas:
                newdf.loc[df[coluna] < 0, coluna] = None
            print(tabulate(newdf.groupby([
                'algorithm',
                'instance', 
                'scenario', 
                'alpha_time', 
                'alpha_cost', 
                'alpha_security', 
                'alpha_restricted_candidate_list',
                'num_buckets'
            ])[colunas_desejadas].mean()))
        print('Fim do scenario', scenario)
    print('Fim da instância', instance)
    print()
print('FIM')

# print(data.groupby(by='Algorithm').sum())
# data.groupby(['Algorithm', 'Instance', 'Scenario', 'Time alpha', 'Cost alpha', 'Security alpha'])

# print(data)
# print(tabulate(data))

# x axis values
# x = [1,2,3]
# corresponding y axis values
# y = [2,4,1]
  
# plotting the points 
# plt.plot(x, y)
  
# naming the x axis
# plt.xlabel('x - axis')
# naming the y axis
# plt.ylabel('y - axis')
  
# giving a title to my graph
# plt.title('My first graph!')
  
# function to show the plot
# plt.show()

In [59]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import figure
from tabulate import tabulate

df = pd.read_csv(f'{path_to_project}/{output_folder}/{selected_option}/{output_file_name}', header='infer')

newdf = df[(1==1)
           & (df['algorithm'] == algorithm)
           # & (df.alpha_time == 0.90) 
           # & (df.alpha_cost == .05) 
           # & (df.alpha_security == .05) 
           & (df['instance'].str.startswith(instance))
           & (df.scenario == scenario)
           & (df.num_buckets == 16)
          ]

# Ordenar o DataFrame de forma decrescente pela coluna 'time'
df_sorted = newdf.sort_values(by='time', ascending=False)

# Selecione apenas as colunas numéricas
colunas_numericas = df_sorted.select_dtypes(include=['float64', 'int64']).columns
# print(colunas_numericas)
colunas_desejadas = [
    'num_iterations',
    'object_function',
    'makespan',
    'cost',
    'security',
    'time',
    'best_solution_iteration',
    'best_solution_time',
    'lsn_time_1',
    'lsn_noi_1',
    'lsn_time_2',
    'lsn_noi_2',
    'lsn_time_3',
    'lsn_noi_3'
]
print(' '.join(map(str, colunas_desejadas)))
# Filtra os valores positivos em cada coluna desejada
# for coluna in colunas_desejadas:
#     newdf[coluna] = newdf[coluna].apply(lambda x: x if x >= 0 else None)
for coluna in colunas_desejadas:
    df_sorted.loc[df[coluna] < 0, coluna] = None
print(tabulate(df_sorted.groupby([
    'algorithm',
    'instance', 
    'scenario', 
    'alpha_time', 
    'alpha_cost', 
    'alpha_security', 
    'alpha_restricted_candidate_list',
    'num_buckets'
])[colunas_desejadas].mean()))


num_iterations object_function makespan cost security time best_solution_iteration best_solution_time lsn_time_1 lsn_noi_1 lsn_time_2 lsn_noi_2 lsn_time_3 lsn_noi_3



In [94]:
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate

# Suponha que você já tenha definido as variáveis necessárias
# path_to_project, output_folder, selected_option, output_file_name, algorithm, instance, scenario

df = pd.read_csv(f'{path_to_project}/{output_folder}/{selected_option}/{output_file_name}', header='infer')
newdf = df[(1==1)
           & (df['num_buckets'] == 16)]
# print(newdf)
# Ordenar o DataFrame de forma decrescente pela coluna 'time'
# df_sorted = newdf.sort_values(by='time', ascending=False)

# Selecionar apenas as colunas desejadas
colunas_desejadas = [
    'num_iterations', 
    'object_function', 
    'makespan', 
    'cost', 
    'security', 
    'time',
    'best_solution_iteration', 
    'best_solution_time', 
    'lsn_time_1', 
    'lsn_noi_1',
    'lsn_time_2', 
    'lsn_noi_2', 
    'lsn_time_3', 
    'lsn_noi_3'
]

# Limpar valores negativos
for coluna in colunas_desejadas:
    newdf.loc[newdf[coluna] < 0, coluna] = None

# Agrupar e calcular a média
media_por_grupo = newdf.groupby([
    'algorithm', 'instance', 'scenario', 'alpha_time', 'alpha_cost', 'alpha_security',
    'alpha_restricted_candidate_list', 'num_buckets'])[colunas_desejadas].mean()

# Imprimir a tabela usando tabulate
# Apelidos para os cabeçalhos das colunas
apelidos = {
    "num_iterations": "n_iter", 
    "object_function": "O.F.", 
    "makespan": "mspn",
    "cost": "cost",
    "security": "security",
    "time": "time",
    "makespan": "mspn",
    "best_solution_iteration": "bsi",
    "best_solution_time": "bst",
    "lsn_time_1": "lsnt_1",
    "lsn_noi_1": "lsni_1",
    "lsn_time_2": "lsnt_2",
    "lsn_noi_2": "lsni_2",
    "lsn_time_3": "lsnt_3",
    "lsn_noi_3": "lsni_3",
}
# Renomear as colunas
media_por_grupo = media_por_grupo.rename(columns=apelidos)
df_sorted = media_por_grupo.sort_values(by='time', ascending=False)
print(tabulate(df_sorted, headers='keys', tablefmt='psql'))


+-----------------------------------------------------+----------+-----------+----------+----------+------------+---------+---------+----------+----------+----------+-----------+----------+----------+----------+
|                                                     |   n_iter |      O.F. |     mspn |     cost |   security |    time |     bsi |      bst |   lsnt_1 |   lsni_1 |    lsnt_2 |   lsni_2 |   lsnt_3 |   lsni_3 |
|-----------------------------------------------------+----------+-----------+----------+----------+------------+---------+---------+----------+----------+----------+-----------+----------+----------+----------|
| ('grasp', 'Sipht_100', 4, 0.05, 0.9, 0.05, 0.3, 16) | 11.1429  | 0.430952  | 12931.4  | 1255.36  | 0.00766786 | 4836.67 | 6.14286 | 2973.3   |  4358.44 |  1966.43 | 0.430952  |  6       |  77.1023 |  601.857 |
| ('grasp', 'Sipht_100', 2, 0.05, 0.05, 0.9, 0.3, 16) | 10.25    | 0.0929199 |  9627.12 |  728.625 | 0.00253863 | 4047.1  | 5.25    | 2297.5   |  3660.1